In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/w2v-model/word2vec_trainingdata_all_size25.model
/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/ratings_small.csv
/kaggle/input/the-movies-dataset/links.csv


In [2]:
#import packages for later use
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from gensim.models import word2vec
from keras.preprocessing.text import text_to_word_sequence
from scipy.spatial import distance

In [3]:
#input dataset
X = pd.read_csv("../input/the-movies-dataset/movies_metadata.csv")
overview = X['overview']
name_overview = X[['original_title','overview','id']]

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
overview.describe()

count                  44512
unique                 44307
top       No overview found.
freq                     133
Name: overview, dtype: object

In [5]:
overview.head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [6]:
overview.shape

(45466,)

In [7]:
print('The size of training sample that we use for trianing is {}'.format(overview.shape[0]))

The size of training sample that we use for trianing is 45466


In [8]:
#prepare the corpus
sentence=[]
for i in overview:
    sentence.append(text_to_word_sequence(str(i),filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n',lower=True,split=" "))

In [9]:
# w2v = word2vec.Word2Vec(sentence,vector_size=25,window=25,min_count=1,workers=5,sg=1,hs=1,sorted_vocab=1,seed=42)
# w2v.save("word2vec_trainingdata_all_size25.model")

In [10]:
w2v = word2vec.Word2Vec.load("../input/w2v-model/word2vec_trainingdata_all_size25.model")

In [11]:
w2v.wv.most_similar('murder', topn=10)

[('murderer', 0.9479272961616516),
 ('accused', 0.9181045293807983),
 ('murdered', 0.9108615517616272),
 ('witnesses', 0.903934121131897),
 ('clammed', 0.9032635688781738),
 ('suspected', 0.9014613032341003),
 ('case', 0.9005158543586731),
 ('smugly', 0.8988717794418335),
 ('kostenko', 0.8976795673370361),
 ('pled', 0.8955886363983154)]

In [12]:
def sen2vec(sentence):
    res = []
    for i in range(len(sentence)):
        vec = np.zeros(25)
        for j in sentence[i]:
            vec += w2v.wv[j]
        vec = vec/len(sentence[i])
        res.append(vec)   
    return res
res = sen2vec(sentence)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


In [13]:
def similarity_movie(name,top,name_overview,contvec):
    index = name_overview.loc[name_overview['original_title'] == name].index[0]
    ans = np.zeros(len(res))
    ans += 1000
    for i in range(len(res)):
        if(np.isfinite(res[i]).all()):
            ans[i] = distance.euclidean(contvec[index],res[i])
    sim = np.argsort(ans)[1:top+1]
    for i in sim:
        print('Title: ',name_overview['original_title'].iloc[i])
        print('Overview: ',name_overview['overview'].iloc[i])
        print('\n')

In [14]:
similarity_movie('Batman',5,name_overview,res)

Title:  The Dark Knight
Overview:  Batman raises the stakes in his war on crime. With the help of Lt. Jim Gordon and District Attorney Harvey Dent, Batman sets out to dismantle the remaining criminal organizations that plague the streets. The partnership proves to be effective, but they soon find themselves prey to a reign of chaos unleashed by a rising criminal mastermind known to the terrified citizens of Gotham as the Joker.


Title:  Batman: Under the Red Hood
Overview:  Batman faces his ultimate challenge as the mysterious Red Hood takes Gotham City by firestorm. One part vigilante, one part criminal kingpin, Red Hood begins cleaning up Gotham with the efficiency of Batman, but without following the same ethical code.


Title:  The Dark Knight Rises
Overview:  Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police Department. Eight years lat